# Introducción a Gymnasium en google colab

## Comentarios

Este notebook ha sido adaptado del material original de la clase T81-558: Applications of Deep Neural Networks, Module 12: Reinforcement Learning, del
Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)

Otros notebooks del mismo autor sobre RL, cuyos enlaces originales son:

* **Part 12.1: Introduction to Introduction to Gymnasium** [[Video]](https://www.youtube.com/watch?v=FvuyrpzvwdI&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_12_1_reinforcement.ipynb)
* Part 12.2: Introduction to Q-Learning [[Video]](https://www.youtube.com/watch?v=VKuqvbG_KAw&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_12_2_qlearningreinforcement.ipynb)
* Part 12.3: Stable Baselines Q-Learning [[Video]](https://www.youtube.com/watch?v=kl7zsCjULN0&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_12_3_pytorch_reinforce.ipynb)
* Part 12.4: Atari Games with Stable Baselines Neural Networks [[Video]](https://www.youtube.com/watch?v=maLA1_d4pzQ&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_12_4_atari.ipynb)
* Part 12.5: Future of Reinforcement Learning [[Video]](https://www.youtube.com/watch?v=-euo5pTjP8E&list=PLjy4p-07OYzuy_lHcRW8lPTLPTTOmUpmi) [[Notebook]](t81_558_class_12_5_rl_future.ipynb)


# 1 - Introducción a Gymnasium

[Gymnasium](https://github.com/Farama-Foundation/Gymnasium) tiene como objetivo proporcionar un punto de referencia de inteligencia general fácil de configurar con diversos entornos. El objetivo es estandarizar la definición de entornos en las publicaciones de investigación en IA para facilitar la reproducción de la investigación publicada. El proyecto afirma ofrecer al usuario una interfaz sencilla. Gymnasium es una bifurcación de OpenAI Gym, para el cual OpenAI dejó de dar soporte en octubre de 2021. Actualmente, Gymnasium cuenta con el soporte de [The Farama Foundation](https://farama.org/).

Gymnasium se instala mediante pip en su equipo local. Hay algunas limitaciones importantes que debe tener en cuenta:

* Gymnasium Atari solo es compatible **directamente** con Linux y Macintosh
* Gymnasium Atari se puede utilizar con Windows; Sin embargo, requiere un procedimiento de instalación específico (https://towardsdatascience.com/how-to-install-openai-gym-in-a-windows-environment-338969e24d30).
* Gymnasium no puede renderizar juegos animados directamente en Google CoLab.

Dado que Gymnasium requiere una pantalla gráfica, la única forma de visualizar Gymnasium en Google CoLab es mediante un vídeo incrustado. La presentación de las animaciones del juego Gymnasium en Google CoLab se describe más adelante en este módulo.

## Análisis de los entornos de Gymnasium

El componente central de Gymnasium es el entorno, que define el "juego" en el que competirá el algoritmo de refuerzo. Un entorno no tiene por qué ser un juego; sin embargo, describe las siguientes características similares a las de un juego:
* **Espacio de acción**: ¿Qué acciones podemos realizar en el entorno en cada paso/episodio para modificarlo?
* **Espacio de observación**: ¿Cuál es el estado actual de la parte del entorno que podemos observar? Normalmente, podemos ver el entorno completo.

Antes de comenzar a analizar Gymnasium, es fundamental comprender la terminología que utiliza esta biblioteca.

* **Agente**: El programa o modelo de aprendizaje automático que controla las acciones.
Paso: Una ronda de acciones que afectan al espacio de observación.
* **Episodio**: Un conjunto de pasos que finaliza cuando el agente no cumple el objetivo del entorno o el episodio alcanza el número máximo de pasos permitidos.
* **Renderizado**: Gymnasium puede renderizar un fotograma para su visualización después de cada episodio.
* **Recompensa**: Un refuerzo positivo que puede ocurrir al final de cada episodio, después de que el agente actúe.
* **No determinista**: En algunos entornos, la aleatoriedad es un factor que determina el efecto de las acciones en la recompensa y los cambios en el espacio de observación.

Gymnasium debe instalarse con el siguiente comando.

In [ ]:
!pip install gymnasium[accept-rom-license,atari]

In [ ]:
!pip install ale-py

Es importante tener en cuenta que muchos entornos Gymnasium especifican que no son no deterministas, aunque utilizan números aleatorios para procesar acciones. Según el issue tracker de biblioteca Gymnasium en GitHub, una propiedad no determinista significa que un entorno determinista se comporta de forma aleatoria. Incluso asignando al entorno un valor de semilla consistente, este comportamiento se confirma.

El programa puede usar el método de semilla de un entorno (`env.seed`) para generar el generador de números aleatorios correspondiente.

La biblioteca Gymnasium permite consultar algunos de estos atributos de los entornos. Creé la siguiente función para consultar los entornos Gymnasium.

In [ ]:
import gymnasium as gym

def query_environment(name):
    env = gym.make(name)
    spec = gym.spec(name)
    print(f"Action Space: {env.action_space}")
    print(f"Observation Space: {env.observation_space}")
    print(f"Max Episode Steps: {spec.max_episode_steps}")
    print(f"Nondeterministic: {spec.nondeterministic}")
# Access reward_range from the spec object if it exists
    if hasattr(spec, 'reward_range'):
        print(f"Reward Range: {spec.reward_range}")
    else:
        print("Reward Range: Not specified in environment spec")

    if hasattr(spec, 'reward_threshold'):
         print(f"Reward Threshold: {spec.reward_threshold}")
    else:
        print("Reward Threshold: Not specified in environment spec")

Analizaremos el entorno **MountainCar-v0**, que desafía a un coche de baja potencia a escapar del valle entre dos montañas. El siguiente código describe el entorno Mountian Car.

In [ ]:
query_environment("MountainCar-v0")

Action Space: Discrete(3)
Observation Space: Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
Max Episode Steps: 200
Nondeterministic: False
Reward Range: Not specified in environment spec
Reward Threshold: -110.0


Este entorno permite tres acciones distintas: acelerar hacia adelante, desacelerar o retroceder. El espacio de observación contiene dos valores continuos (de punto flotante), como lo demuestra el objeto de caja. El espacio de observación es simplemente la posición y la velocidad del coche. El coche tiene 200 pasos para escapar en cada episodio. Habría que consultar el código, pero el coche de montaña no recibe ninguna recompensa incremental. La única recompensa para el vehículo se obtiene al escapar del valle.

In [ ]:
query_environment("CartPole-v1")

Action Space: Discrete(2)
Observation Space: Box([-4.8               -inf -0.41887903        -inf], [4.8               inf 0.41887903        inf], (4,), float32)
Max Episode Steps: 500
Nondeterministic: False
Reward Range: Not specified in environment spec
Reward Threshold: 475.0


El entorno **CartPole-v1** reta al agente a equilibrar un poste mientras el agente. También pueden conocer este problema con el nombre de péndulo invertido. El entorno tiene un espacio de observación de 4 números continuos:

* Posición del carro
* Velocidad del carro
* Ángulo del poste
* Velocidad del poste en la punta

Para lograr este objetivo, el agente puede realizar las siguientes acciones:

* Empujar el carro hacia la izquierda
* Empujar el carro hacia la derecha

También existe una variante continua del carro de montaña. Esta versión no solo tiene el motor encendido o apagado. El espacio de acción es un único número de punto flotante para el carro continuo que especifica cuánta fuerza hacia adelante o hacia atrás utiliza actualmente.

In [ ]:
query_environment("MountainCarContinuous-v0")

Action Space: Box(-1.0, 1.0, (1,), float32)
Observation Space: Box([-1.2  -0.07], [0.6  0.07], (2,), float32)
Max Episode Steps: 999
Nondeterministic: False
Reward Range: Not specified in environment spec
Reward Threshold: 90.0


Gymnasium ofrece una plataforma versátil para desarrollar y comparar algoritmos de aprendizaje por refuerzo. Es compatible con una amplia gama de entornos, incluyendo juegos clásicos de Atari, a través del emulador Arcade Learning Environment (ALE). Esta integración permite a investigadores y entusiastas acceder a una colección de videojuegos retro diseñados originalmente para la consola Atari 2600, utilizándolos como referencia para el rendimiento de la IA. Al interactuar con ALE, los usuarios de Gymnasium pueden implementar fácilmente sus algoritmos y probarlos frente a los desafíos que presentan estos juegos. Cada juego presenta escenarios únicos que ayudan a entrenar algoritmos para que aprendan diversas tareas, lo que convierte a Gymnasium en una herramienta invaluable para el avance de la inteligencia artificial a través de estos entornos interactivos y complejos.

Los algoritmos de aprendizaje por refuerzo (RL) pueden recibir información de un juego de Atari de dos maneras principales, que se adaptan a diferentes aspectos del estado y la complejidad del juego.

El primer método consiste en monitorizar la pantalla del juego o la salida visual que genera. En este enfoque, el algoritmo de RL procesa los píxeles de la pantalla del juego como el estado de su entorno. Esto es similar a cómo un jugador humano vería e interpretaría el juego. El algoritmo analiza los patrones, movimientos y cambios dentro de los fotogramas para tomar decisiones sobre la mejor acción a realizar en cada paso. Este método requiere que el modelo de aprendizaje por retroalimentación (RL) maneje datos de alta dimensión y aprenda a asociar las señales visuales con los resultados del juego.

El segundo método consiste en monitorizar la RAM del sistema Atari. A pesar de su capacidad limitada, la RAM de un sistema Atari contiene toda la información sobre el estado interno del juego, como la ubicación de los objetos, las puntuaciones de los jugadores y el estado del juego. Al acceder directamente a esta memoria, un algoritmo de aprendizaje por retroalimentación (RL) puede acceder a una representación más compacta y con menos ruido del estado del juego que la que proporcionan los datos de píxeles. Esto puede ser beneficioso para un aprendizaje más eficiente, ya que el estado del sistema se representa de forma más estructurada y con menos dimensiones.

Ambos métodos tienen sus ventajas. El enfoque de captura de pantalla obliga al algoritmo a aprender directamente de la información visual, lo cual es un enfoque más general y más cercano a cómo los humanos juegan. Por otro lado, el método de monitorización de RAM puede resultar en tiempos de entrenamiento más rápidos y, potencialmente, en una comprensión más profunda de la mecánica del juego, ya que evita la necesidad de interpretar datos visuales. La elección entre estos métodos depende de los objetivos y las limitaciones específicas del proyecto de aprendizaje directo en cuestión.

Primero, veamos cómo monitorizar la pantalla del juego [Breakout](https://ale.farama.org/environments/breakout/).

In [ ]:
import ale_py

gym.register_envs(ale_py)

env = gym.make('ALE/Breakout-v5')
query_environment("ALE/Breakout-v5")

Action Space: Discrete(4)
Observation Space: Box(0, 255, (210, 160, 3), uint8)
Max Episode Steps: None
Nondeterministic: False
Reward Range: Not specified in environment spec
Reward Threshold: None


De manera similar, podemos observar Breakout RAM

In [ ]:
query_environment("ALE/Breakout-ram-v5")

NameNotFound: Environment `Breakout-ram` doesn't exist in namespace ALE. Did you mean: `Breakout`?

# 2-  Render OpenAI Gym Environments from CoLab

It is possible to visualize the game your agent is playing, even on CoLab. This section provides information on generating a video in CoLab that shows you an episode of the game your agent is playing. I based this video process on suggestions found [here](https://colab.research.google.com/drive/1flu31ulJlgiRL1dnN2ir8wGh9p7Zij2t).

Begin by installing **pyvirtualdisplay** and **python-opengl**.

In [ ]:
# HIDE OUTPUT
!pip install pyvirtualdisplay
!sudo apt-get install -y xvfb ffmpeg

Next, we install the needed requirements to display an Atari game.

In [ ]:
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install -U renderlab

In [ ]:
!pip install gymnasium[all]
!pip install moviepy  # Para procesamiento de video
!apt update &> /dev/null
!apt install xvfb &> /dev/null  # Para renderizado sin pantalla
!apt install python-opengl &> /dev/null
!apt install ffmpeg &> /dev/null  # Para codificación de video

## Recordvideo tutorial

In [ ]:
import gymnasium as gym
from gymnasium.wrappers import RecordVideo
import numpy as np

# Crear el ambiente base
env = gym.make("CartPole-v1", render_mode="rgb_array")

# Envolver con RecordVideo
# video_folder: carpeta donde se guardarán los videos
# episode_trigger: función que decide cuándo grabar (por defecto cada 10 episodios)
env = RecordVideo(
    env,
    video_folder="./videos",
    episode_trigger=lambda episode_id: episode_id % 10 == 0,  # Grabar cada 10 episodios
    name_prefix="cartpole-training"
)

# Entrenar un agente simple (random)
num_episodes = 50

for episode in range(num_episodes):
    obs, info = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Política random para demostración
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        total_reward += reward
        done = terminated or truncated

    print(f"Episodio {episode}: Reward = {total_reward}")

env.close()
print("Videos guardados en la carpeta ./videos")

In [ ]:
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay


# Display the video
list_of_files = glob.glob('videos/*.mp4')

# Check if any video files were generated
if list_of_files:
    video_path = list_of_files[0]
    video = io.open(video_path, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''
        <video width="640" height="480" controls>
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        </video>
    '''.format(encoded.decode('ascii'))))
else:
    print("No video file was generated.")

## notebook original setup video

Note, the above cell may request to restart the runtime, if this occurs, please restart the CoLab runtime. Next, we define the functions used to show the video by adding it to the CoLab notebook.

Now we are ready to play the game.  We use a simple random agent.

In [ ]:
import gym
import gymnasium as gymnasium
from gymnasium.wrappers import RecordVideo
import glob
import io
import base64
from IPython.display import HTML
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display
# from renderlab import Recorder
from colabgymrender.recorder import Recorder

# Start virtual display
display = Display(visible=0, size=(1400, 900))
display.start()

# Create environment
#
env = gymnasium.make('ALE/Breakout-v5',render_mode="rgb_array")
directory = './video'
env = Recorder(env, directory)
# env = Recorder(env, frame_rate=30)

# env = gymnasium.make('ALE/Breakout-v5', render_mode="rgb_array")
# env.metadata['render_fps'] = 30
# Reset the environment
env.reset()

# Setup the wrapper to record the video
# Record every episode
video_callable=lambda episode_id: True
env = RecordVideo(env, video_folder='./videos', episode_trigger=video_callable)

# Run the environment until done
terminated = False
truncated = False
while not (terminated or truncated):
    action = env.action_space.sample()  # replace with your own policy!
    obs, reward, terminated, truncated, info = env.step(action)

env.close()



In [ ]:
# Display the video
list_of_files = glob.glob('videos/*.mp4')

# Check if any video files were generated
if list_of_files:
    video_path = list_of_files[0]
    video = io.open(video_path, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''
        <video width="640" height="480" controls>
            <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        </video>
    '''.format(encoded.decode('ascii'))))
else:
    print("No video file was generated.")

You will note that the **step** and **reset** functions return several values:

* **observation** (ObsType): An element of the environment's observation_space as the next observation due to the agent actions. An example is a numpy array containing the positions and velocities of the pole in CartPole.

* **reward** (SupportsFloat): The reward as a result of taking the action.

* **terminated** (bool): Whether the agent reaches the terminal state (as defined under the MDP of the task) which can be positive or negative. An example is reaching the goal state or moving into the lava from the Sutton and Barton, Gridworld. If true, the user needs to call reset().

* **truncated** (bool): Whether the truncation condition outside the scope of the MDP is satisfied. Typically, this is a timelimit, but could also be used to indicate an agent physically going out of bounds. Can be used to end the episode prematurely before a terminal state is reached. If true, the user needs to call reset().

* **info** (dict): Contains auxiliary diagnostic information (helpful for debugging, learning, and logging). This might, for instance, contain: metrics that describe the agent's performance state, variables that are hidden from observations, or individual reward terms that are combined to produce the total reward.